## Web scraping 

Goal : get a table of all the locations listed [here](https://en.wikipedia.org/wiki/Lists_of_World_Heritage_Sites), in the format \[url; name; coordinates\]

In [1]:
import requests
from bs4 import BeautifulSoup
import random
import pandas as pd
import lxml
import unicodedata2 as unicodedata

In [2]:
response = requests.get(url="https://en.wikipedia.org/wiki/Lists_of_World_Heritage_Sites")

soup = BeautifulSoup(response.content, 'html.parser')

title = soup.find(id="firstHeading")

# Get all the links
allLinks = soup.find(id="bodyContent").find_all("a")


link_list = []
str_link = str(allLinks).split('<a')

for link in str_link:
    if 'href="/wiki/' in link:
        link_list.append(link)
    

In [3]:
print(allLinks)

[<a class="mw-jump-link" href="#mw-head">Jump to navigation</a>, <a class="mw-jump-link" href="#searchInput">Jump to search</a>, <a class="image" href="/wiki/File:Welterbe.svg"><img alt="" class="thumbimage" data-file-height="160" data-file-width="160" decoding="async" height="150" src="//upload.wikimedia.org/wikipedia/commons/thumb/2/24/Welterbe.svg/150px-Welterbe.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/2/24/Welterbe.svg/225px-Welterbe.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/2/24/Welterbe.svg/300px-Welterbe.svg.png 2x" width="150"/></a>, <a class="internal" href="/wiki/File:Welterbe.svg" title="Enlarge"></a>, <a href="/wiki/Logo" title="Logo">Logo</a>, <a href="/wiki/UNESCO" title="UNESCO">UNESCO</a>, <a href="/wiki/World_Heritage_Committee" title="World Heritage Committee">World Heritage Committee</a>, <a href="/wiki/World_Heritage_Site" title="World Heritage Site">World Heritage Site</a>, <a href="/wiki/UNESCO" title="UNESCO">United Natio

In [5]:
links1 = []
for elem in soup.select("a[href*=wiki\/List_of]"):
    links1.append(elem['href'])

In [6]:
# create a csv file
with open('../data/unesco_sites_raw.csv', 'w') as f:

    for link in links1:
        response = requests.get(url = 'https://en.wikipedia.org' + link)

        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find('table',{'class':"wikitable"})
        df = pd.read_html(str(table))

        # convert list to dataframe
        df = pd.DataFrame(df[0])
        newdf = df.fillna('')

        # drop the unwanted column
        if 'Site' in newdf.columns and 'Location' in newdf.columns:
            data = newdf[['Site', 'Location']]

            # add information to the csv file
            for index, value in data.iterrows():
                location_cleaned = unicodedata.normalize('NFKD', value['Location']).encode('ascii','ignore').decode('ascii').replace(';', ',')
                f.write("%s;%s;%s\n" % (link, value['Site'], location_cleaned))


NameError: name 'unicodedata' is not defined

### Clean the data 
Clean the text related to place entities. 

In [ ]:
import pandas as pd
import csv
import re
from tqdm.autonotebook import tqdm
from Levenshtein import distance as lev 

In [ ]:
data = pd.read_csv('../data/unesco_sites_raw.csv', delimiter=';', header = None)
data.rename(columns = {0: 'link', 1: 'place', 2: 'location'}, inplace = True)

In [ ]:
data['place'] = data['place'].apply(lambda x: x.replace("-"," "))
data['place'] = data['place'].apply(lambda x: re.sub(r'[^A-Za-z0-9 ]+', '', x))
data['place'] = data['place'].apply(lambda x: x.strip())
data['place'] = data['place'].apply(lambda x: x.capitalize())

In [7]:
len(data), len(data.place.unique())

AttributeError: 'DataFrame' object has no attribute 'place'

In [8]:
data.head()

,Site,Location
0,Aapravasi Ghat,"Port Louis District, Mauritius.mw-parser-outpu..."
1,Abu Mena,"Abusir, Egypt30°50′28″N 29°39′47″E﻿ / ﻿30.8409..."
2,Air and Ténéré Natural Reserves,"Arlit Department, Niger18°N 9°E﻿ / ﻿18°N 9°E"
3,Aksum,"Tigray Region, Ethiopia14°07′49″N 38°43′07″E﻿ ..."
4,Al Qal'a of Beni Hammad,"Maadid, Algeria35°49′06″N 4°47′13″E﻿ / ﻿35.818..."


In [9]:
for i in tqdm(range(len(data) - 1)):
    place1 = data.iloc[i].place
    place2 = data.iloc[i+1].place
    if lev(place1, place2) < 3: # if the writing of the place differs of less than 2 characters
        data.iloc[i+1].place = place1 # only the first version of the writing is kept


NameError: name 'tqdm' is not defined

In [10]:
data.drop_duplicates(subset = 'place', keep='first',inplace=True)
data.sort_values('place', ascending=True, inplace = True)

KeyError: Index(['place'], dtype='object')

In [11]:
# check that all names of places are unique
len(data), len(data.place.unique())

AttributeError: 'DataFrame' object has no attribute 'place'

In [12]:
# save the dataframe 
data.to_csv('../data/unesco_sites_cleaned.csv', sep = ';', header = False, index = False)

## Video search

Get all the links resulting from the youtube requests. Refer to video-search.py for the python script.

In [13]:
import numpy as np
import youtube_dl
import csv
import urllib
import urllib.request
import re
import unidecode
from tqdm.autonotebook import tqdm
import pytube
import pandas as pd
from pytube import YouTube

/tmp/ipykernel_763812/1353721322.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [14]:
inputfile = csv.reader(open('../data/unesco_sites_cleaned.csv','r'), delimiter=';')

ydl_opts = {
'format': 'bestaudio/best',
'outtmpl': 'tmp/%(id)s.%(ext)s',
'noplaylist': True,
'quiet': True,
'prefer_ffmpeg': True,
'audioformat': 'wav',
'forceduration':True
}

# create an empty dictionnary
video_dict = {}

In [15]:

# read line by line
for row in inputfile:
    
    print(row)
    # get second column (names of places)
    place = row[1]
    
    # clean string : remove accents
    place_clean1 = unidecode.unidecode(place)
    # clean string : remove spaces
    place_clean2 = place_clean1.replace(' ', '+')
    
    # add key words 
    search_words = place_clean2 + "+drone"
    
    # make a request in youtube, store the results in a list
    results = []
    html = urllib.request.urlopen("https://www.youtube.com/results?search_query=" + search_words)
    
    # store the results
    video_ids = re.findall(r"watch\?v=(\S{11})", html.read().decode())
  
    for video_id in tqdm(video_ids):
        try:
            ydl_opts = {'ignoreerrors': True}
            with youtube_dl.YoutubeDL(ydl_opts) as ydl:
                myVideo = YouTube("https://www.youtube.com/watch?v=%s" % video_id)
                if (myVideo.streams.filter(res="2160p") != None) :
                    if not myVideo.age_restricted:
                        dictMeta = ydl.extract_info("https://www.youtube.com/watch?v=%s" % video_id, download=False)
            
                        video_dict.update({video_id : [place_clean1, dictMeta['duration'], dictMeta['title'], dictMeta['upload_date']]})
            
        except Exception as e:
            print("ERROR Catched and Passed", e)
            pass 
        
    
video_df = pd.DataFrame.from_dict(video_dict, orient='index', columns=['place', 'duration', 'title', 'date'])

video_df.to_csv('../data/video_info.csv', index_label = 'id')

['Aapravasi Ghat', 'Port Louis District,\xa0Mauritius.mw-parser-output .geo-default,.mw-parser-output .geo-dms,.mw-parser-output .geo-dec{display:inline}.mw-parser-output .geo-nondefault,.mw-parser-output .geo-multi-punct{display:none}.mw-parser-output .longitude,.mw-parser-output .latitude{white-space:nowrap}20°09′31″S 57°30′11″E\ufeff / \ufeff20.158611°S 57.503056°E']


  0%|          | 0/32 [00:00<?, ?it/s]

[youtube] _CU1pndJgQw: Downloading webpage
[youtube] _CU1pndJgQw: Downloading MPD manifest
[youtube] gKRtzTDmTiE: Downloading webpage
[youtube] gKRtzTDmTiE: Downloading MPD manifest
[youtube] EgvQojibIfc: Downloading webpage
[youtube] EgvQojibIfc: Downloading MPD manifest



KeyboardInterrupt



## Screenshots

Select randomly 10 places, from which select 10 random videos, and take 3 screenshots for each video without downloading it for analysis purposes

In [ ]:
import pandas as pd
import random
import youtube_dl
import numpy
import cv2

In [ ]:
df = pd.read_csv('../data/video_info.csv', delimiter=',')
dict_sample = {}
places_sel = random.sample(df.place.unique().tolist(), 10)
for place in places_sel:
    ids = df.loc[df['place'] == place].id.tolist()
    ids_sample = random.sample(ids, 10)
    dict_sample.update({place : ids_sample})

In [ ]:
for key, value in dict_sample.items():
    
    place = key
    for youtube_id in value:
        print(youtube_id)
        video_url = 'https://www.youtube.com/watch?v=%s' % youtube_id  #The Youtube URL
        ydl_opts = {}
        ydl = youtube_dl.YoutubeDL(ydl_opts)
        info_dict = ydl.extract_info(video_url, download=False)

        formats = info_dict.get('formats',None)

        f = formats[-1]
        url = f.get('url',None)
        cap = cv2.VideoCapture(url)

        x = 1
        count = info_dict['duration'] / 4

        while x < 4 :

            for i in range(3):

                time = x * info_dict['duration'] + ( 10 * i ) # each time, take 3 shots at 10 sec intervals
                ret, frame = cap.read()
                if not ret:
                    break
                filename = "../data/screenshots/%s-%d.png" % (youtube_id, time)

                cv2.imwrite(filename, frame)
                count += int(info_dict['duration'] / 4)
                cap.set(1,count)

            x += 1

        cap.release()